<a href="https://colab.research.google.com/github/rotogni/diffusion-lagr/blob/master/diff_lagr_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Wed Mar 26 08:08:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   44C    P8             12W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Set up environment for guided diffusion project
!apt-get update
!apt-get install -y libopenmpi-dev openmpi-bin
!pip install mpi4py
!CC=mpicc CXX=mpicxx pip install --no-binary=h5py h5py

# Clone the repository (replace with the actual repo URL)
!git clone https://github.com/rotogni/diffusion-lagr.git
%cd diffusion-lagr

# Install the package in development mode
!pip install -e .

# Verify installation
!python -c "import guided_diffusion; print('guided_diffusion installed successfully')"

import os
os.makedirs("/content/model_checkpoints", exist_ok=True)
os.makedirs("/content/logfiles", exist_ok=True)

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,381 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [4,041 kB]
Get:13 https://r2u.stat.illinois.edu/u

In [ ]:
# Set the flags
DATA_FLAGS="--dataset_path datasets/trajectories.h5 --dataset_name train"
MODEL_FLAGS="--dims 1 --image_size 300 --in_channels 3 --num_channels 32 --num_res_blocks 2 --attention_resolutions 150,75 --channel_mult 1,2,4"
DIFFUSION_FLAGS="--diffusion_steps 800 --noise_schedule tanh6,1"
TRAIN_FLAGS="--lr 1e-4 --batch_size 8 --lr_anneal_steps 500000 --save_interval 10000"

In [ ]:
# Training command
!python scripts/turb_train.py $DATA_FLAGS $MODEL_FLAGS $DIFFUSION_FLAGS $TRAIN_FLAGS

Streaming output truncated to the last 5000 lines.
| mse_q0     | 0.208    |
| mse_q1     | 0.0761   |
| mse_q2     | 0.0405   |
| mse_q3     | 0.0226   |
| param_norm | 86.9     |
| samples    | 1.42e+04 |
| step       | 1.78e+03 |
-------------------------
-------------------------
| grad_norm  | 1.01     |
| loss       | 0.0691   |
| loss_q0    | 0.163    |
| loss_q1    | 0.0686   |
| loss_q2    | 0.0316   |
| loss_q3    | 0.0197   |
| mse        | 0.0691   |
| mse_q0     | 0.163    |
| mse_q1     | 0.0686   |
| mse_q2     | 0.0316   |
| mse_q3     | 0.0197   |
| param_norm | 86.9     |
| samples    | 1.43e+04 |
| step       | 1.79e+03 |
-------------------------
-------------------------
| grad_norm  | 1.21     |
| loss       | 0.102    |
| loss_q0    | 0.23     |
| loss_q1    | 0.0879   |
| loss_q2    | 0.0433   |
| loss_q3    | 0.0202   |
| mse        | 0.102    |
| mse_q0     | 0.23     |
| mse_q1     | 0.0879   |
| mse_q2     | 0.0433   |
| mse_q3     | 0.0202   |
| param_norm 

In [ ]:
# Set sampling flags
SAMPLE_FLAGS="--num_samples 16 --batch_size 8 --model_path /content/model_checkpoints/ema_0.9999_009900.pt"

#Please note that the $MODEL_FLAGS and $DIFFUSION_FLAGS should be the same as those used in training.

In [ ]:
# Sampling command
!python scripts/turb_sample.py $SAMPLE_FLAGS $MODEL_FLAGS $DIFFUSION_FLAGS


Logging to /tmp/openai-2025-03-25-19-26-21-512080
creating model and diffusion...
sampling...
created 8 samples
created 16 samples
saving to /content/logfiles/samples_16x300x3.npz
sampling complete
[rank0]:[W325 19:26:57.040904716 ProcessGroupNCCL.cpp:1496] Warning: WARNING: destroy_process_group() was not called before program exit, which can leak resources. For more info, please see https://pytorch.org/docs/stable/distributed.html#shutdown (function operator())


In [ ]:
# After sampling with the above command, it will generate a file named samples_179200x2000x3.npz (for DM-3c as an example).
# You can use the following code to read and retrieve the generated velocities:

import h5py
import numpy as np

with h5py.File('datasets/trajectories.h5', 'r') as h5f:
    rx0 = np.array(h5f.get('min'))
    rx1 = np.array(h5f.get('max'))

u3c = (np.load('/content/logfiles/samples_16x300x3.npz')['arr_0']+1)*(rx1-rx0)/2 + rx0
# save u3c as a file
np.save('/content/logfiles/u3c_samples_16x300x3.npy', u3c)
